
# Hyperparameter search

After having selected number of promising feature set candidates in Notebook 4, we perform similar validation experiments in order to select optimal hyperparameters for GBDT (CatBoost) and random forest regressors.

In these validation experiments, we use the same training and vaidation sets as in Notebook 4: use December 2014 for validation when training with all the preceding months. Similarly, clipped RMSE is used as the performance metric.

In the experiments grid searches are performed over plausible hyperparameter spaces of the respective classifiers. The search is performed separately for each of the five feature subsets with 9, 16 or 25 features selected.


In [1]:
import sys
import os.path
import json
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
from itertools import product
import gc
from tqdm import tqdm_notebook
import re
from catboost import CatBoostRegressor, Pool
from utils import load_feature_set, clipped_rmse, HoldOut
import pickle

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive') 
  if not os.path.isfile('SETTINGS.json'):
       # hard coded data directory in drive is used if SETTINGS.json not present 
       config={}
       config['DATA_DIR'] = '/content/gdrive/My Drive/kaggle-c1'
       with open('SETTINGS.json', 'w') as outfile:
         json.dump(config, outfile)

with open('SETTINGS.json') as config_file:
    config = json.load(config_file)

DATA_DIR = config['DATA_DIR']

print('Using DATA_DIR ', DATA_DIR)

DATA_FOLDER = DATA_DIR

Using DATA_DIR  C:\Users\ville\easema\repos\c1-final\datadir


In [2]:
def iterate_parameter_combinations(options):
    keys = options.keys()
    values = (options[key] for key in keys)
    return [dict(zip(keys, combination)) for combination in product(*values)]

In [3]:
# read feature selection results from disk

with open(os.path.join(DATA_FOLDER,'search_path_sfs.pickle'), 'rb') as fp:    
            search_path_sfs = pickle.load(fp)

In [4]:
numsel=3
id = 'allfeat'
list(search_path_sfs[id][numsel]['feature_names'])

['target_lag_2', 'item_category_id', 'target_category_tfidf_unigram_256_lag_5']

In [2]:
!pip install catboost

  Using cached https://files.pythonhosted.org/packages/44/ef/beae4b4ef80902f22e3af073397f079c96969c69b2c7d52a57ea9ae61c9d/retrying-1.3.3.tar.gz
  Stored in directory: C:\Users\ville\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


## CatBoost regression

In [ ]:
grid = {'learning_rate': [0.03, 0.1,0.3,0.5],
        'iterations': [100,300,1000],
        'depth': [10,12,14,15,16],
        'l2_leaf_reg': [0.3, 1, 3, 10],
          'task_type': ['GPU']}
try:
    with open(os.path.join(DATA_FOLDER,'best_params_catboost.json')) as fp:
        best_params=json.load(fp)           
except:        
    best_params={}
    
X_train, y_train, X_trainval, y_trainval, X_val, y_val, X_test, submissionidx2testidx = load_feature_set('allfeat', data_folder=DATA_FOLDER)

for id in ['basic','basicv2','allfeat','text','within']:
  for numsel in [9,16,25]: 
      if '{}_sel{}'.format(id,numsel) in list(best_params.keys()):
        print('skipping already searched feature set {}'.format('{}_sel{}'.format(id,numsel)))
        continue;
    
      
      comb_list=iterate_parameter_combinations(grid)
      score_list=[]
      for param in comb_list:
        print('Feature set {}_sel{} combination {}/{}'.format(id,numsel,len(score_list)+1,len(comb_list))) 
        print('keeping cols ', list(search_path_sfs[id][numsel]['feature_names']))
        to_drop_cols= [col for col in X_train.columns.values if not col in list(search_path_sfs[id][numsel]['feature_names'])]
        reg=CatBoostRegressor(metric_period=100, **param)
        reg.fit(X_train.drop(to_drop_cols,axis=1).to_numpy(), y_train)
        pred_val = np.clip(reg.predict(X_val.drop(to_drop_cols,axis=1).to_numpy()), 0, 20)
        rmse = clipped_rmse(y_val, pred_val)
        print('Feature set {} params {} Clipped RMSE {}'.format(id,param,rmse))
        score_list.append(rmse)
        del reg
        gc.collect()
        
      print('** Best CatBoostRegressor params for feature set {}_{}: {}'.format(id,numsel,comb_list[np.argmin(score_list)]))      
      best_params['{}_sel{}'.format(id,numsel)]=comb_list[np.argmin(score_list)]
      with open(os.path.join(DATA_FOLDER,'best_params_catboost.json'), 'w') as fp:
        json.dump(best_params, fp)  
        

reading from file /home/vvi/repos/kaggle-final-project/datadir/feature_set_allfeat.csv
skipping already searched feature set basic_sel9
skipping already searched feature set basic_sel16
skipping already searched feature set basic_sel25
skipping already searched feature set basicv2_sel9
skipping already searched feature set basicv2_sel16
skipping already searched feature set basicv2_sel25
skipping already searched feature set allfeat_sel9
skipping already searched feature set allfeat_sel16
skipping already searched feature set allfeat_sel25
skipping already searched feature set text_sel9
Feature set text_sel16 combination 1/240
keeping cols  ['target_category_tfidf_unigram_256_lag_2', 'date_block_num', 'item_category_id', 'target_category_frequent_256_within_shop_lag_5', 'target_category_tfidf_bigram_256_within_shop_lag_2', 'target_category_tfidf_unigram_256_within_shop_lag_12', 'item_id', 'target_category_tfidf_bigram_256_within_shop_lag_6', 'target_category_frequent_256_lag_5', 'targe

## Random forest regression

In [5]:
from sklearn.ensemble import RandomForestRegressor

grid = {'max_depth': [10,12,14,16],
       'n_estimators': [30,50,100]}


try:
    with open(os.path.join(DATA_FOLDER,'best_params_catboost.rf')) as fp:
        best_params=json.load(fp)           
except:        
    best_params={}
    

best_params={}          

X_train, y_train, X_trainval, y_trainval, X_val, y_val, X_test, submissionidx2testidx = load_feature_set('allfeat', data_folder=DATA_FOLDER)

for id in ['basic','basicv2','allfeat','text','within']:
  for numsel in [9,16,25]:  
     if '{}_sel{}'.format(id,numsel) in list(best_params.keys()):
        print('skipping already searched feature set {}'.format('{}_sel{}'.format(id,numsel)))
        continue;
    
      comb_list=iterate_parameter_combinations(grid)
      score_list=[]
      for param in comb_list:
        print('Feature set {}_sel{} combination {}/{}'.format(id,numsel,len(score_list)+1,len(comb_list))) 
        print('param dict: ', param)
        print('keeping cols ', list(search_path_sfs[id][numsel]['feature_names']))
        to_drop_cols= [col for col in X_train.columns.values if not col in list(search_path_sfs[id][numsel]['feature_names'])]
        reg=RandomForestRegressor(verbose=2, n_jobs=-1, **param)
        reg.fit(X_train.drop(to_drop_cols,axis=1).to_numpy(), y_train)
        pred_val = np.clip(reg.predict(X_val.drop(to_drop_cols,axis=1).to_numpy()), 0, 20)
        rmse = clipped_rmse(y_val, pred_val)
        print('Feature set {} params {} Clipped RMSE {}'.format(id,param,rmse))
        score_list.append(rmse)
        del reg
        gc.collect()
        
      print('** Best RandomForestRegressor params for feature set {}_{}: {}'.format(id,numsel,comb_list[np.argmin(score_list)]))      
      best_params['{}_sel{}'.format(id,numsel)]=comb_list[np.argmin(score_list)]
      with open(os.path.join(DATA_FOLDER,'best_params_rf.json'), 'w') as fp:
        json.dump(best_params, fp)  
        

building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50building tree 19 of 50

building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50building tree 34 of 50

building tree 35 of 50
building tree 36 of 50


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.1min


building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  4.4min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.2s finished


Feature set within params {'max_depth': 12, 'n_estimators': 50} Clipped RMSE 0.5493221283567645
Feature set within_sel16 combination 6/12
param dict:  {'max_depth': 12, 'n_estimators': 100}
keeping cols  ['target_category_tfidf_bigram_256_within_shop_lag_12', 'target_category_within_shop_lag_4', 'date_block_num', 'item_category_id', 'target_category_tfidf_unigram_256_within_shop_lag_2', 'target_category_tfidf_bigram_256_within_shop_lag_4', 'target_category_tfidf_unigram_256_within_shop_lag_6', 'target_category_frequent_256_within_shop_lag_4', 'target_category_within_shop_lag_3', 'target_category_tfidf_bigram_256_within_shop_lag_5', 'target_category_tfidf_unigram_256_within_shop_lag_5', 'target_category_frequent_256_within_shop_lag_12', 'shop_id', 'target_category_frequent_256_within_shop_lag_3', 'target_category_tfidf_bigram_256_within_shop_lag_3', 'target_category_tfidf_unigram_256_within_shop_lag_3']


...

building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100



building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  6.3min


building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100building tree 47 of 100

building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100building tree 54 of 100

building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 17.8min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.7s finished


Feature set within params {'max_depth': 16, 'n_estimators': 100} Clipped RMSE 0.541868218579768
** Best RandomForestRegressor params for feature set within_25: {'max_depth': 14, 'n_estimators': 50}
